In [9]:
import numpy as np

from qiskit.circuit.library import TwoLocal
from qiskit import QuantumCircuit
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.quantum_info import SparsePauliOp

from qiskit_addon_cutting import partition_problem
from qiskit_addon_cutting import generate_cutting_experiments

from qiskit_aer import AerSimulator
from qiskit_ibm_runtime import SamplerV2, Batch

In [129]:
# constructing quantum circuit of same structure as given in the qasm file

num_qubits = 20
reps = 4
gate_sequence = ['rz','sx','rz','sx','x','rz']

# nearest neighbour configuration
seq_pairs = [(i,(i+1)%num_qubits) for i in range(0,num_qubits,2)]
warp_pairs = [(i,(i-1)%num_qubits) for i in range(0,num_qubits,2)]

# qc two local
qc = TwoLocal(num_qubits,rotation_blocks=['rz','sx','rz','sx','x','rz'],entanglement_blocks='cz', entanglement=[seq_pairs,warp_pairs], reps=2,skip_final_rotation_layer=True).decompose()  
qc.assign_parameters([0.41] * len(qc.parameters), inplace=True)
#qc.measure_all()
qc.draw(scale=0.8)

┌──────────┐┌────┐┌──────────┐┌────┐┌───┐┌──────────┐   ┌──────────┐»
 q_0: ┤ Rz(0.41) ├┤ √X ├┤ Rz(0.41) ├┤ √X ├┤ X ├┤ Rz(0.41) ├─■─┤ Rz(0.41) ├»
      ├──────────┤├────┤├──────────┤├────┤├───┤├──────────┤ │ ├──────────┤»
 q_1: ┤ Rz(0.41) ├┤ √X ├┤ Rz(0.41) ├┤ √X ├┤ X ├┤ Rz(0.41) ├─■─┤ Rz(0.41) ├»
      ├──────────┤├────┤├──────────┤├────┤├───┤├──────────┤   ├──────────┤»
 q_2: ┤ Rz(0.41) ├┤ √X ├┤ Rz(0.41) ├┤ √X ├┤ X ├┤ Rz(0.41) ├─■─┤ Rz(0.41) ├»
      ├──────────┤├────┤├──────────┤├────┤├───┤├──────────┤ │ ├──────────┤»
 q_3: ┤ Rz(0.41) ├┤ √X ├┤ Rz(0.41) ├┤ √X ├┤ X ├┤ Rz(0.41) ├─■─┤ Rz(0.41) ├»
      ├──────────┤├────┤├──────────┤├────┤├───┤├──────────┤   ├──────────┤»
 q_4: ┤ Rz(0.41) ├┤ √X ├┤ Rz(0.41) ├┤ √X ├┤ X ├┤ Rz(0.41) ├─■─┤ Rz(0.41) ├»
      ├──────────┤├────┤├──────────┤├────┤├───┤├──────────┤ │ ├──────────┤»
 q_5: ┤ Rz(0.41) ├┤ √X ├┤ Rz(0.41) ├┤ √X ├┤ X ├┤ Rz(0.41) ├─■─┤ Rz(0.41) ├»
      ├──────────┤├────┤├──────────┤├────┤├───┤├──────────┤   ├──────────┤»
 q_6: ┤ Rz(0.41) ├┤ √X ├┤ Rz(0.41) ├┤ √X ├┤ X ├┤ Rz(0.41) ├─■─┤ Rz(0.41) ├»
      ├──────────┤├────┤├──────────┤├────┤├───┤├──────────┤ │ ├──────────┤»
 q_7: ┤ Rz(0.41) ├┤ √X ├┤ Rz(0.41) ├┤ √X ├┤ X ├┤ Rz(0.41) ├─■─┤ Rz(0.41) ├»
      ├──────────┤├────┤├──────────┤├────┤├───┤├──────────┤   ├──────────┤»
 q_8: ┤ Rz(0.41) ├┤ √X ├┤ Rz(0.41) ├┤ √X ├┤ X ├┤ Rz(0.41) ├─■─┤ Rz(0.41) ├»
      ├──────────┤├────┤├──────────┤├────┤├───┤├──────────┤ │ ├──────────┤»
 q_9: ┤ Rz(0.41) ├┤ √X ├┤ Rz(0.41) ├┤ √X ├┤ X ├┤ Rz(0.41) ├─■─┤ Rz(0.41) ├»
      ├──────────┤├────┤├──────────┤├────┤├───┤├──────────┤   ├──────────┤»
q_10: ┤ Rz(0.41) ├┤ √X ├┤ Rz(0.41) ├┤ √X ├┤ X ├┤ Rz(0.41) ├─■─┤ Rz(0.41) ├»
      ├──────────┤├────┤├──────────┤├────┤├───┤├──────────┤ │ ├──────────┤»
q_11: ┤ Rz(0.41) ├┤ √X ├┤ Rz(0.41) ├┤ √X ├┤ X ├┤ Rz(0.41) ├─■─┤ Rz(0.41) ├»
      ├──────────┤├────┤├──────────┤├────┤├───┤├──────────┤   ├──────────┤»
q_12: ┤ Rz(0.41) ├┤ √X ├┤ Rz(0.41) ├┤ √X ├┤ X ├┤ Rz(0.41) ├─■─┤ Rz(0.41) ├»
      ├──────────┤├────┤├──────────┤├────┤├───┤├──────────┤ │ ├──────────┤»
q_13: ┤ Rz(0.41) ├┤ √X ├┤ Rz(0.41) ├┤ √X ├┤ X ├┤ Rz(0.41) ├─■─┤ Rz(0.41) ├»
      ├──────────┤├────┤├──────────┤├────┤├───┤├──────────┤   ├──────────┤»
q_14: ┤ Rz(0.41) ├┤ √X ├┤ Rz(0.41) ├┤ √X ├┤ X ├┤ Rz(0.41) ├─■─┤ Rz(0.41) ├»
      ├──────────┤├────┤├──────────┤├────┤├───┤├──────────┤ │ ├──────────┤»
q_15: ┤ Rz(0.41) ├┤ √X ├┤ Rz(0.41) ├┤ √X ├┤ X ├┤ Rz(0.41) ├─■─┤ Rz(0.41) ├»
      ├──────────┤├────┤├──────────┤├────┤├───┤├──────────┤   ├──────────┤»
q_16: ┤ Rz(0.41) ├┤ √X ├┤ Rz(0.41) ├┤ √X ├┤ X ├┤ Rz(0.41) ├─■─┤ Rz(0.41) ├»
      ├──────────┤├────┤├──────────┤├────┤├───┤├──────────┤ │ ├──────────┤»
q_17: ┤ Rz(0.41) ├┤ √X ├┤ Rz(0.41) ├┤ √X ├┤ X ├┤ Rz(0.41) ├─■─┤ Rz(0.41) ├»
      ├──────────┤├────┤├──────────┤├────┤├───┤├──────────┤   ├──────────┤»
q_18: ┤ Rz(0.41) ├┤ √X ├┤ Rz(0.41) ├┤ √X ├┤ X ├┤ Rz(0.41) ├─■─┤ Rz(0.41) ├»
      ├──────────┤├────┤├──────────┤├────┤├───┤├──────────┤ │ ├──────────┤»
q_19: ┤ Rz(0.41) ├┤ √X ├┤ Rz(0.41) ├┤ √X ├┤ X ├┤ Rz(0.41) ├─■─┤ Rz(0.41) ├»
      └──────────┘└────┘└──────────┘└────┘└───┘└──────────┘   └──────────┘»
«      ┌────┐┌──────────┐┌────┐┌───┐┌──────────┐      
« q_0: ┤ √X ├┤ Rz(0.41) ├┤ √X ├┤ X ├┤ Rz(0.41) ├────■─
«      ├────┤├──────────┤├────┤├───┤├──────────┤    │ 
« q_1: ┤ √X ├┤ Rz(0.41) ├┤ √X ├┤ X ├┤ Rz(0.41) ├─■──┼─
«      ├────┤├──────────┤├────┤├───┤├──────────┤ │  │ 
« q_2: ┤ √X ├┤ Rz(0.41) ├┤ √X ├┤ X ├┤ Rz(0.41) ├─■──┼─
«      ├────┤├──────────┤├────┤├───┤├──────────┤    │ 
« q_3: ┤ √X ├┤ Rz(0.41) ├┤ √X ├┤ X ├┤ Rz(0.41) ├─■──┼─
«      ├────┤├──────────┤├────┤├───┤├──────────┤ │  │ 
« q_4: ┤ √X ├┤ Rz(0.41) ├┤ √X ├┤ X ├┤ Rz(0.41) ├─■──┼─
«      ├────┤├──────────┤├────┤├───┤├──────────┤    │ 
« q_5: ┤ √X ├┤ Rz(0.41) ├┤ √X ├┤ X ├┤ Rz(0.41) ├─■──┼─
«      ├────┤├──────────┤├────┤├───┤├──────────┤ │  │ 
« q_6: ┤ √X ├┤ Rz(0.41) ├┤ √X ├┤ X ├┤ Rz(0.41) ├─■──┼─
«      ├────┤├──────────┤├────┤├───┤├──────────┤    │ 
« q_7: ┤ √X ├┤ Rz(0.41) ├┤ √X ├┤ X ├┤ Rz(0.41) ├─■──┼─
«      ├──

In [130]:
observable = SparsePauliOp(["Z"*num_qubits])

part_labels = "A"*(num_qubits//2)+"B"*(num_qubits//2)

partitioned_problem = partition_problem(
    circuit=qc, partition_labels=part_labels,observables=observable.paulis)

subcircuits = partitioned_problem.subcircuits
subobservables = partitioned_problem.subobservables
bases = partitioned_problem.bases

print('circuit-1:',subcircuits["A"].draw(scale=0.8))
print('circuit-2:',subcircuits["B"].draw(scale=0.8))

print(f"Sampling overhead: {np.prod([basis.overhead for basis in bases])}")

circuit-1:        ┌──────────┐┌────┐┌──────────┐┌────┐┌───┐┌──────────┐   ┌──────────┐»
q29_0: ┤ Rz(0.41) ├┤ √X ├┤ Rz(0.41) ├┤ √X ├┤ X ├┤ Rz(0.41) ├─■─┤ Rz(0.41) ├»
       ├──────────┤├────┤├──────────┤├────┤├───┤├──────────┤ │ ├──────────┤»
q29_1: ┤ Rz(0.41) ├┤ √X ├┤ Rz(0.41) ├┤ √X ├┤ X ├┤ Rz(0.41) ├─■─┤ Rz(0.41) ├»
       ├──────────┤├────┤├──────────┤├────┤├───┤├──────────┤   ├──────────┤»
q29_2: ┤ Rz(0.41) ├┤ √X ├┤ Rz(0.41) ├┤ √X ├┤ X ├┤ Rz(0.41) ├─■─┤ Rz(0.41) ├»
       ├──────────┤├────┤├──────────┤├────┤├───┤├──────────┤ │ ├──────────┤»
q29_3: ┤ Rz(0.41) ├┤ √X ├┤ Rz(0.41) ├┤ √X ├┤ X ├┤ Rz(0.41) ├─■─┤ Rz(0.41) ├»
       ├──────────┤├────┤├──────────┤├────┤├───┤├──────────┤   ├──────────┤»
q29_4: ┤ Rz(0.41) ├┤ √X ├┤ Rz(0.41) ├┤ √X ├┤ X ├┤ Rz(0.41) ├─■─┤ Rz(0.41) ├»
       ├──────────┤├────┤├──────────┤├────┤├───┤├──────────┤ │ ├──────────┤»
q29_5: ┤ Rz(0.41) ├┤ √X ├┤ Rz(0.41) ├┤ √X ├┤ X ├┤ Rz(0.41) ├─■─┤ Rz(0.41) ├»
       ├──────────┤├────┤├──────────┤├────┤├───┤├──────────┤   ├─

In [131]:
subexperiments, coefficients = generate_cutting_experiments(
    circuits=subcircuits, observables=subobservables, num_samples=np.inf
)

In [132]:
backend = AerSimulator()

pass_manager = generate_preset_pass_manager(optimization_level=1, backend=backend)
isa_subexperiments = {
    label: pass_manager.run(partition_subexpts)
    for label, partition_subexpts in subexperiments.items()
}

with Batch(backend=backend) as batch:
    sampler = SamplerV2(mode=batch)
    jobs = {
        label: sampler.run(subsystem_subexpts, shots=2**12).result()
        for label, subsystem_subexpts in isa_subexperiments.items()
    }

In [133]:
jobs

{'A': PrimitiveResult([SamplerPubResult(data=DataBin(observable_measurements=BitArray(<shape=(), num_shots=4096, num_bits=10>), qpd_measurements=BitArray(<shape=(), num_shots=4096, num_bits=1>)), metadata={'shots': 4096, 'circuit_metadata': {}}), SamplerPubResult(data=DataBin(observable_measurements=BitArray(<shape=(), num_shots=4096, num_bits=10>), qpd_measurements=BitArray(<shape=(), num_shots=4096, num_bits=1>)), metadata={'shots': 4096, 'circuit_metadata': {}}), SamplerPubResult(data=DataBin(observable_measurements=BitArray(<shape=(), num_shots=4096, num_bits=10>), qpd_measurements=BitArray(<shape=(), num_shots=4096, num_bits=1>)), metadata={'shots': 4096, 'circuit_metadata': {}}), SamplerPubResult(data=DataBin(observable_measurements=BitArray(<shape=(), num_shots=4096, num_bits=10>), qpd_measurements=BitArray(<shape=(), num_shots=4096, num_bits=1>)), metadata={'shots': 4096, 'circuit_metadata': {}}), SamplerPubResult(data=DataBin(observable_measurements=BitArray(<shape=(), num_sho

In [134]:
result_A = jobs['A']
#result_A

result_A[0].data.observable_measurements.get_counts()

{'0000000000': 1002,
 '1100000001': 2,
 '0001000000': 146,
 '0100000000': 162,
 '0111001000': 1,
 '0000000010': 165,
 '0000000100': 139,
 '0000000110': 29,
 '0000110000': 11,
 '0110000110': 2,
 '0000011001': 4,
 '0010000000': 143,
 '1000000000': 148,
 '0010001000': 20,
 '0000000001': 155,
 '0000001000': 167,
 '1000100001': 10,
 '0000001001': 26,
 '0000001010': 28,
 '0010000100': 22,
 '0000100100': 27,
 '0000100000': 149,
 '0000101100': 2,
 '0010000010': 22,
 '1000100100': 6,
 '0101000110': 2,
 '0010000001': 23,
 '0100001001': 6,
 '0011000000': 10,
 '1000010000': 20,
 '1001000000': 27,
 '0000011000': 27,
 '0100000010': 26,
 '0100100010': 6,
 '0000010000': 161,
 '0101001000': 5,
 '0001100000': 19,
 '0100000100': 24,
 '0100001000': 29,
 '0100100000': 25,
 '0110110000': 1,
 '1010000001': 4,
 '0010110000': 2,
 '0000101000': 27,
 '0001001010': 1,
 '0010010001': 4,
 '0101000000': 30,
 '1001011000': 1,
 '1101010100': 1,
 '0101100101': 1,
 '1000000001': 21,
 '0110000000': 20,
 '1100000000': 11,

In [135]:
result_B = jobs['B']
#result_B

result_B[0].data.observable_measurements.get_counts()

{'0000001000': 144,
 '0000010000': 147,
 '0000000010': 158,
 '0000000000': 966,
 '0000000100': 175,
 '0000100001': 22,
 '0101000110': 1,
 '1000000000': 149,
 '0010000000': 158,
 '0010100000': 36,
 '0001000000': 159,
 '0000100000': 126,
 '0000000110': 24,
 '0010001000': 24,
 '0010010100': 6,
 '0000010010': 22,
 '1001000010': 1,
 '1000100001': 3,
 '1000010000': 31,
 '0100000000': 155,
 '1000000100': 24,
 '0001001001': 7,
 '0001000001': 9,
 '0100100000': 25,
 '1001000100': 4,
 '0001000100': 36,
 '0101000000': 28,
 '0000101000': 16,
 '0000001001': 33,
 '0110100000': 6,
 '1100000100': 4,
 '0001001000': 30,
 '0000010100': 19,
 '0000000001': 155,
 '0000001101': 2,
 '0000011010': 5,
 '0001001011': 1,
 '0110010000': 3,
 '0110000000': 17,
 '0001100110': 1,
 '0000001110': 2,
 '0000000101': 35,
 '0110100001': 1,
 '0100000001': 23,
 '1010101000': 4,
 '0001011000': 5,
 '1010000000': 24,
 '1000000001': 29,
 '0100000100': 27,
 '1000010010': 6,
 '1000000010': 29,
 '1000100000': 18,
 '0010000001': 25,
 

In [136]:
counts = {}




In [137]:
## Testing the output of original circuit
qc.measure_all()
sampler = SamplerV2(mode=backend)
result = sampler.run([qc],shots=2**12).result()

In [138]:
result[0].data.meas.get_counts()

{'00010000001000100000': 2,
 '10000000001100000100': 2,
 '00000000000000100000': 36,
 '00000000000010000100': 6,
 '10000000000001101000': 1,
 '00000000000000001000': 38,
 '00001000000000000100': 6,
 '00000000000000000000': 227,
 '00001000000110010000': 2,
 '00000000010000000001': 5,
 '00000010000000000100': 6,
 '00000001000000000001': 9,
 '00000000000100010000': 11,
 '01000000000001000000': 9,
 '10000000101010100110': 1,
 '00010101000010010000': 1,
 '00000000000000000100': 47,
 '10100000000100000000': 1,
 '00000000010100000000': 8,
 '00000010001000001000': 2,
 '01000001000000001001': 1,
 '00000000001010000000': 5,
 '00100000100000000000': 8,
 '00000000011000010100': 1,
 '10000000000000000000': 41,
 '00000000000001000000': 41,
 '01010000000000000011': 1,
 '00100010000000011010': 1,
 '00000000100000001010': 1,
 '00000010000000000001': 7,
 '00100001100001100010': 1,
 '00000000000000000001': 41,
 '00000000101000000000': 8,
 '00000000100100000000': 6,
 '01000000000000000010': 6,
 '000100001